<a href="https://colab.research.google.com/github/Mingjia000/Master_thesis/blob/main/preference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import nn
import numpy as np
from scipy.special import softmax

In [ ]:
class nnPolicy(nn.Module):
    
    def __init__(self, input_size, output_size):
      #input_size= feature,output_size=out_feature
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, output_size),
        )



    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        x = softmax(x)
        return x

In [ ]:
class nnReward(nn.Module):
    
    def __init__(self, input_size, output_size):
      #input_size= feature,output_size=out_feature
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, output_size),
        )



    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

In [ ]:
def segment_attribute(s,time_interval,cost_matrix,connection_matrix):
  #input: segment [state, state,state]; state=[activation, location]; 1,2,3,4,
  #output: attribue maxtrix [cost, time]
  # cost, time, reliability, emissions, risk of damage
  route=np.zeros(cost_matrix.shape)
  _s= s[s[:,0] == 1] #activation=1,activated
  _s= _s[:,1] #terminal ID

  _s_origin=_s[:-1]
  _s_destinaiton=_s[1:]

  route[_s_origin,_s_destinaiton]=1 #passed links

  cost=np.sum(route*cost_matrix)
  connection=np.sum(connection_matrix*route)

  time= len(s)* time_interval

  attribute=[connection, cost, time] #connection=0,full connected

  return attribute


In [16]:
import numpy as np
arr = np.array([[1,1],[0,1],[0,2],[1,2],[1,3]])
arr=arr[arr [:,0] == 1]
arr=arr[:,1]
route=np.zeros((5,5))
route[arr,arr]=1
print(route)

[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]]


In [ ]:
def masked_reward_function (c):
     #input: c attribute of a segment
     #output: the ground true reward
     r=c[0] #only cost
     return r

In [ ]:
def reward_interface(s1,s2,time_interval,cost_matrix):
  #input: two segment
  #output: preference [0,1],[1,0],[0.5,0.5],[can not compare]
  #c1,c2: attribute of segments
  c1=segment_attribute(s1,time_interval,cost_matrix) 
  c2=segment_attribute(s2,time_interval,cost_matrix)
  r1_true= masked_reward_function(c1)    # self-defined, only value tcost
  r2_true= masked_reward_function(c2)
  if r1_true>r2_true:
      pref=[1,0]
  elif r1_true<r2_true:
      pref=[0,1]
  else:
      pref=[0.5,0.5] 
  '''
  if c1[0]==0 and c2[0]!=0:#segment越短越好？
    pref=[1,0]
  elif  c1[0]!=0 and c2[0]==0:
    pref=[0,1]
  elif  c1[0]==0 and c2[0]==0:
    if r1_true>r2_true:
      pref=[1,0]
    elif r1_true<r2_true:
      pref=[0,1]
    else:
      pref=[0.5,0.5]
  #else:
  #  print('can not compare')
   '''

  return pref

In [ ]:
def process_policy(output,connection_matrix):
  output=output[connection_matrix==0]
  id=np.arange(len(output))
  output_id=output[id==0]
  action=output_id[np.argmax(output)]
  return action

In [ ]:
input_size_policy= 2 # [current location, the connected info]，[[activation, current terminal]
output_size_policy= 6 # the node number
input_size_reward_predictor= #(current state, current action)
output_size_reward_predictor= 1
initial_reward=

#state:activation, location
#activation:next station
'''
find the optimal policy
if activation=1, policy is implemented
  next state is determined  [activation, same terminal]
else:
  determin the next state [activation, same terminal]

find the optimal reward
if the policy give a impossible route--hard reward function
else: ask for feedback

'''

#define the network
time_matrix=np.array([[0,1,2,0,0,0],
                      [2,0,0,1,2,0],
                      [2,0,0,2,2,0],
                      [0,2,2,0,0,1],
                      [0,2,2,0,0,2],
                      [0,0,0,2,2,0]])

cost_matrix=np.array([[0,2,1,0,0,0],
                      [2,0,0,2,2,0],
                      [2,0,0,2,1,0],
                      [0,2,2,0,0,2],
                      [0,2,2,0,0,1],
                      [0,0,0,2,2,0]])
#mode_matrix=
connection_matrix=np.array([[0,0,0,1,1,1],
                      [0,0,1,0,0,1],
                      [0,1,0,0,0,1],
                      [1,0,0,0,1,0],
                      [1,0,0,1,0,0],
                      [1,1,1,0,0,0]])
#connection=0,full connected

#intial reward, train the policy, exmine connection, give feedback, update reward
